![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Load the CSV file
df = pd.read_csv("rental_info.csv")

# Convert rental_date and return_date to datetime format
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

# Calculate rental length in days
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# Create dummy variables for special_features
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in str(x) else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in str(x) else 0)

# Drop columns that leak target or are irrelevant
X = df.drop(columns=['rental_length_days', 'return_date', 'rental_date', 'special_features'])

# Define target variable
y = df['rental_length_days']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Try Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_preds)

# Try Random Forest Regressor
rf_model = RandomForestRegressor(random_state=9)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_preds)

# Choose the best model based on MSE (and MSE < 3 requirement)
if rf_mse < lr_mse and rf_mse < 3:
    best_model = rf_model
    best_mse = rf_mse
elif lr_mse < 3:
    best_model = lr_model
    best_mse = lr_mse
else:
    best_model = rf_model if rf_mse < lr_mse else lr_model
    best_mse = min(rf_mse, lr_mse)

print("Best model:", best_model)
print("Best MSE on test set:", best_mse)


Best model: RandomForestRegressor(random_state=9)
Best MSE on test set: 2.030141907417274
